# Speech recognition for isolated words

Date: 30/11/2020

Name: Jiangnan HUANG, Jingzhuo HUI, You ZUO


In [1]:
import doctest
import struct
import wave
from glob import glob

import numpy as np
from matplotlib import pyplot as plt
from numpy import fft

In [2]:
def readwav(fname):
    """
    Parameters
    ----------
    fname : str
        The filename of the wave file to process

    Returns
    -------
    (x, fs) : tuple

    x : np.array
        The 1-dim. PCM audio signal

    fs : int
        The sampling frequency
    """
    with wave.open(fname) as wav:
        n = wav.getnframes()
        frames = wav.readframes(n)
        x = struct.unpack("h" * n, frames)
        fs = wav.getframerate()
    return np.array(x), fs

In [3]:
# wav, fs = readwav("digits/train/ae_1a.wav")

1 or 2 lines of code for each function below, no need for any description of your code 

In [4]:
def mel(f):
    """
    Frequency converter from Hz to Mel

    Parameters
    ----------
    f : int, float or np.array
        The frequency (Hz)

    Returns
    -------
    mel : float or np.array
        The frequency (Mel)

    Examples
    --------
    >>> x = np.linspace(200, 4000, 20)
    >>> with np.printoptions(precision=2):
    ...     print(mel(x))
    [  283.23  509.38  697.65  858.93  999.99 1125.34 ... 2146.06]
    """
    return 2595 * np.log10(1+f/700.)


def hertz(m):
    """
    Frequency converter from Mel to Hz

    Parameters
    ----------
    mel : int, float or np.array
        The frequency (Mel)

    Returns
    -------
    f : float or np.array
        The frequency (Hz)

    Examples
    --------
    >>> x = np.linspace(200, 2000, 20)
    >>> with np.printoptions(precision=2):
    ...     print(hertz(x))
    [  135.93  209.24  288.97  375.7   470.04  572.64 ... 3428.68]
    """
    return 700 * (10**(m/2595.)-1)

In [5]:
def melbank(nfilt, win_n, fs):
    """
    Computing of the MFCC coefficients of a whole wave file

    Parameters
    ----------
    nfilt : int
        The number of melbank filters

    win_n : int
        The window size (in samples)

    fs : int
        The sampling frequency (Hz)

    Returns
    -------
    melb : np.array
        The 2-dim. (nfilt, win_n/2+1) mel-bank filters

    Examples
    --------
    >>> melb = melbank(nfilt=24, win_n=256, fs=8000)
    >>> melb.shape
    (24, 129)
    >>> with np.printoptions(precision=2, suppress=True, edgeitems=6):
    ...     print(melb)
    [[0.   1.   0.5  0.   0.   0.   ... 0.   0.   0.   0.   0.   0.  ]
     [0.   0.   0.5  1.   0.5  0.   ... 0.   0.   0.   0.   0.   0.  ]
     [0.   0.   0.   0.   0.5  1.   ... 0.   0.   0.   0.   0.   0.  ]
     [0.   0.   0.   0.   0.   0.   ... 0.   0.   0.   0.   0.   0.  ]
     [0.   0.   0.   0.   0.   0.   ... 0.   0.   0.   0.   0.   0.  ]
     [0.   0.   0.   0.   0.   0.   ... 0.   0.   0.   0.   0.   0.  ]
     ...
     [0.   0.   0.   0.   0.   0.   ... 0.   0.   0.   0.   0.   0.  ]
     [0.   0.   0.   0.   0.   0.   ... 0.   0.   0.   0.   0.   0.  ]
     [0.   0.   0.   0.   0.   0.   ... 0.   0.   0.   0.   0.   0.  ]
     [0.   0.   0.   0.   0.   0.   ... 0.   0.   0.   0.   0.   0.  ]
     [0.   0.   0.   0.   0.   0.   ... 0.   0.   0.   0.   0.   0.  ]
     [0.   0.   0.   0.   0.   0.   ... 0.36 0.27 0.18 0.09 0.   0.  ]]
    >>> with np.printoptions(precision=2, suppress=True, edgeitems=6):
    ...     print(melb[-1])
    [0.   0. ... 0.1  0.2 ... 1.   0.91 0.82 0.73 ... 0.18 0.09 0.   0.  ]
    """
    melb = np.zeros((nfilt, win_n // 2 + 1))
    melstep = mel(fs / 2) / (nfilt + 1)

    for i in range(nfilt):
        left = int(hertz(melstep * i) * win_n / fs)
        middle = int(hertz(melstep * (i + 1)) * win_n / fs)
        right = int(hertz(melstep * (i + 2)) * win_n / fs)

        melb[i, left : middle + 1] = (
            np.arange(middle - left + 1) * 1.0 / (middle - left)
        )
        melb[i, middle : right + 1] = 1.0 - np.arange(
            right - middle + 1
        ) * 1.0 / (right - middle)

    return melb

\<describe here in few lines your implementation of the function below\>

The function `mfcc()` computes for each given frame y, the 12 coeffients of mel-frequenct cepstral and the normalized energy of signal as the 1st coeffients. We chose the peak value of signal to be a normalization term.

In [6]:
def mfcc(y, melb, nc):
    """
    Computing of the MFCC coefficients of a single frame

    Parameters
    ----------
    y : np.array
        The 1-dim. frame signal

    melb: np.array
        The 2-dim. mel-bank filters

    nc: int
        The number of coefficients

    Returns
    -------
    cc : np.array
        The 1-dim cepstral coefficients

    Examples
    --------
    >>> int16 = np.iinfo(np.int16)
    >>> rng = np.random.default_rng(0)
    >>> y = rng.integers(int16.min, int16.max, 256)
    >>> melb = melbank(nfilt=24, win_n=256, fs=8000)
    >>> cc = mfcc(y, melb, nc=13)
    >>> cc.shape
    (13,)
    """
    s = np.abs(fft.rfft(y))     # dim: (129,) = 256/2 + 1
    E = melb @ s    # (24, 129) @ (129,1) = (24,1)

    # Cepstrum coefficients
    cc = fft.irfft(np.log(E + 1e-20))[1:nc] 

    # energy e
    e = y**2
    e[e <= 1e-20] = 1e-20
    e0 = max(np.log(e))
    e = np.log(sum(e)) - e0

    return np.append(e,cc)

\<describe here in few lines your implementation of the function below\>

We want to have both delta and delta delta as features, so we compute them with function `delta()` in a more general definition:
$$d_t = \frac{\sum_{n=1}^N n(c_{t+n} - c_{t-n})}{2\sum_{n=1}^{N} n^2}$$

where $d_t$ refers to the delta coefficient for frame $t$, and we have $N=1$ for delta and $N=2$ for delta delta features.

In [7]:
def delta(coefficients,N=1):
    """
    Computing of delta of N degree

    Parameters
    ----------
    coefficients : np.array
        The 2-dim. (n_frames, nc) cepstral coefficients of the whole file
    N : int 
        The differential degree of delta, can only be 1 or 2
        Returns
    -------
    c : np.array
        The 2-dim. (n_frames, nc) cepstral differential features of the whole file     
    """
    denominator = 2 * sum([i**2 for i in range(1,N+1)])
    delta_c = np.empty_like(coefficients)
    padded = np.pad(coefficients,pad_width=((N,N),(0,0)), mode='edge')

    for t in range(len(coefficients)):
        delta_c[t] = (np.arange(-N, N+1) @ padded[t : t+2*N+1])/ denominator
    return delta_c

In the function `wav2mfcc()`, for each signal x with its rate fs:

1. firstly implement function `preemph()` to boost the amount of energy in the high frequencies
2. compute the size of each frame(window), the step size for slicing the audio waveform into sliding frames and the number of frames/ windows
3. pad the uncomplete frame with zero
4. get the melbank filters by calling the previous function `melbank()`
5. for each frame, implement the Hamming window multiplication and compute the 13 mfcc 
6. compute the feature delfa and delta delta for all the frames and concatenate all the 39 features


In [8]:
def wav2mfcc(fname, nc=13, nfilt=24, win=0.032, step=0.010):
    """
    Computing of the MFCC coefficients of a whole wave file

    Parameters
    ----------
    fname : str
        The filename of the wave file to process

    nc : int
        The number of cepstral coefficients

    nfilt : int
        The number of melbank filters

    win : float
        The window size (s)

    step : float
        The step size (s)

    Returns
    -------
    c : np.array
        The 2-dim. (n_frames, nc) cepstral coefficients of the whole file
    """
    x, fs = readwav(fname)

    # Pre-emphasis
    x = preemph(x)

    win_n = int(win*fs) # 0.032 * 8000 = 256
    step_n = int(step*fs)   # 0.010 * 8000 = 80
 
    if len(x) <= win_n:
        n_frames = 1
    else:
        n_frames = 1 + int(np.ceil((1.*len(x)-win_n)/step_n))

    padded_len = int((n_frames - 1) * step_n + win_n)
    x = np.concatenate((x, np.zeros(padded_len-len(x))))

    # Cepstral Coefficients
    c = np.empty([n_frames,nc])
    melb = melbank(nfilt=nfilt, win_n=win_n, fs=fs)

    ## Hamming window
    w = np.hamming(win_n)   # (256,)

    for i in range(n_frames):
        s = w * x[step_n*i:step_n*i + win_n] 
        cc = mfcc(s, melb, nc)
        c[i] = cc

    # Feature delta
    d_c = delta(c)

    # delta delta
    dd_c = delta(c,2)

    c = np.hstack((c,d_c,dd_c))
 
    return c

\<describe here in few lines your implementation of the function below\>

Function `euclid()` computes the eclidean distance which is defined as $dist(v1,v2) = \sqrt{\sum_{i=1}^n (v1_i - v2_i)^2}$

In [9]:
def euclid(v1, v2):
    """
    Computation of the Euclidean distance between the MFCC frames

    Parameters
    ----------
    v1 : np.array
        1-dim. MFCC frame

    v2 : np.array
        1-dim. MFCC frame

    Returns
    -------
    dist : np.float
        The Euclidean distance value
    """
    return np.linalg.norm(v1-v2)

\<describe here in few lines your implementation of the function below\>

The implementation of pre-emphasis makes information in higher formants more available to the acoustic model. It's like a kind of smoothing step.

In [10]:
def preemph(x, alpha=0.97):
    """
    (Optional)

    Applying the pre-emphasis step to the PCM audio signal:
    to decrease high frequency energy

    Parameters
    ----------
    x : np.array
        The 1-dim. PCM audio signal

    alpha : int
        The pre-emphasis coefficient

    Returns
    -------
    x_pre : np.array
        The 1-dim. filtered audio signal
    """
    
    return np.append(x[0],x[1:] - alpha * x[:-1])

\<describe here in few lines your implementation of the function below\>

Since we want to compare the similarity between speech signals and find the most correlative one in train set, we need to match the sequences with common trends but may have defferent lengths. 

Algorithm DTW offers a way to calculate an optimal match between two given sequences c1 and c2, where the distance is the value at the right bottom of computed matrix.

In [11]:
def dtw(c1, c2):
    """
    Computation of DTW between MFCC signals

    Parameters
    ----------
    c1 : np.array
        2-dim. MFCC signal

    c1 : np.array
        2-dim. MFCC signal

    Returns
    -------
    dist : np.float
        The DTW distance value
    """
    n, m = c1.shape[0], c2.shape[0]

    g = np.zeros([n+1,m+1])
    g[0,1:] = np.inf
    g[1:,0] = np.inf
    
    for i in range(1,n+1):
        for j in range(1,m+1):
            prec = [g[i-1,j-1],g[i,j-1],g[i-1,j]]
            g[i,j] = min(prec) + euclid(c1[i-1],c2[j-1])
    
    return g[-1,-1]

Write below your computation script

\<describe here in few lines your implementation below\>

For each of the test data, we will find the most "similar" signal in the training dataset. The **similarity** is measured by the *dynamic time warping* of the sequences of 39 MFCC features.

In [12]:
test_filePath = "./digits/test/"
train_filePath = "./digits/train/"

test_files = os.listdir(test_filePath)
train_files = os.listdir(train_filePath)

test_mfcc = []  # lenth = 98
train_mfcc = []
for f in test_files:
    test_mfcc.append(wav2mfcc(test_filePath+f))

for f in train_files:
    train_mfcc.append(wav2mfcc(train_filePath+f))

In [13]:
final_dtw = np.zeros([len(test_files),len(train_files)])
for i in range(len(test_mfcc)):
    for j in range(len(train_mfcc)):
        final_dtw[i,j] = dtw(test_mfcc[i],train_mfcc[j])

In [14]:
min_index = np.argmin(final_dtw, axis=1)

predict_files = []
for i in range(len(test_files)):
    predict_files.append(train_files[min_index[i]])

In [15]:
acc = 0
for i in range(len(test_files)):
    if predict_files[i][3] == test_files[i][3]:
        acc = acc + 1

print(acc)
print(len(test_files))
print(acc/len(test_files))

89
98
0.9081632653061225


Uncomment to test the functions including a doctest in their docstring

In [16]:
doctest.testmod(optionflags=doctest.ELLIPSIS | doctest.NORMALIZE_WHITESPACE)

TestResults(failed=0, attempted=14)

---